In [5]:
import torch.nn.functional as F
from timeit import default_timer
from utilities3 import *
from matplotlib import pyplot as plt
from pdb import set_trace as st
import cv2
from unet_parts import *
from pytorch_msssim import  ms_ssim, ssim
from metrics import *
import wandb
import sys
import json
import resnet

In [9]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, train_F,  transform=None, mean=None, std=None,  eps=1e-10):
        'Initialization'
        with open(train_F, 'r') as openfile: 
            data = json.load(openfile) 
        data = np.array(data)
        #self.labels = data[:,1].astype(int)
        self.data = data[:,0]
        self.targets = data[:,1].astype(float)
        self.mean = mean
        self.std = std
        self.eps = eps

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.data)

    def __getitem__(self, index):
        x = torch.load(self.data[index])
        try:
            x = (x - self.mean) / (self.std + self.eps)
        except:
            print(x.shape)
        return x, self.targets[index]

In [12]:
torch.manual_seed(0)
np.random.seed(0)

N = 700 * 3
ntrain = 1760 #900*2
ntest = 2200-1760#150*2

batch_size = 4
learning_rate = 1e-2#0.001
epochs = 90
iterations = epochs * (ntrain // batch_size)

modes = 20 #12
modes_lat = 12#12
width = 16 #28 #32

r = 10
x_lat = 32#64
x_dep = 955#857
y_lat = 877
y_dep = 844
x_dep_sample = 2

#noise_level = '20dB'
USE_WANDB = True
data_dir1 = '/mlroom/data/lung_set1' #pts/'
data_dir = '/mlroom/data/lung_set1_fix'
EVALUATE = 1-True

mean_file = torch.load('raw_mean.pt')
std_file = torch.load('raw_std.pt')

train_set = Dataset('add_a0_4500_train.json', mean = mean_file, std=std_file)
test_set = Dataset('add_a0_4500_test.json', mean = mean_file, std=std_file)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                            shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size,
                                            shuffle=True)#False)
ntrain = len(train_loader)
ntest = len(test_loader)



In [13]:
model = resnet.resnet18(num_classes=1).cuda()

In [29]:
for raw, area in train_set:
    print(area)

96.13502502441406
59.93937683105469
52.45415115356445
81.18426513671875
79.2904281616211
78.78599548339844
81.77639770507812
51.39179611206055
50.47850036621094
82.55802154541016
26.34527015686035
61.50486373901367
67.68728637695312
25.01547622680664
36.05088424682617
71.18264770507812
51.504783630371094
43.013675689697266
25.757099151611328
94.06881713867188
56.26533126831055
27.26398468017578
23.081483840942383
29.031301498413086
18.865642547607422
27.878799438476562
52.94135284423828
50.99974822998047
28.635448455810547
99.21951293945312
24.07492446899414
28.58356285095215
64.9180908203125
63.79426193237305
77.43196105957031
30.895530700683594
92.55511474609375
21.516902923583984
64.08734130859375
96.02304077148438
21.191967010498047
64.84198760986328
60.028846740722656
46.87213897705078
53.73526382446289
24.685531616210938
46.35829162597656
46.12049865722656
43.012969970703125
64.05949401855469
23.651424407958984
97.47242736816406
10.81183910369873
89.77352905273438
36.520660400390

KeyboardInterrupt: 